# Rede Convolucional e Transfer Learning

### Trabalho 2 da disciplina Noções de Inteligência Artificial - 2/2024
### Alunos: Felipe Lopes Gibin Duarte e Matheus das Neves Fernandes

## Introdução
Neste trabalho usaremos uma rede convolucional pré-treinada (VGG) e a aplicaremos em um problema novo. Experimentaremos com a divisão da base em treinamento, validação e teste, e usaremos validação para o "early stopping" na tentativa de controlar o sobre-ajuste. A base de dados usada é a "TensorFlow Flowers Dataset". Ela contém 3670 imagens coloridas de flores pertencentes a uma de 5 classes: Margarida,
Dente-de-leão, Rosa, Girassol e Tulipa.

## 1. Preparação do Ambiente
Nesta seção, importaremos as bibliotecas necessárias e carregaremos a base de dados.

### 1.1 Importação de bibliotecas 

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## 1.2 Carregamento dos dados

In [2]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
"tf_flowers",
split=["train[:70%]", "train[70%:]"], ## Train test split
batch_size=-1,
as_supervised=True, # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

print (train_ds.shape)
print (test_ds.shape)

2024-12-17 10:29:52.226756: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2024-12-17 10:29:52.440233: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


(2569, 150, 150, 3)
(1101, 150, 150, 3)


## 2. Treinando um MLP
Vamos comprovar que um MLP não apresenta desempenho satisfatório para o problema. Vamos evitar o overfitting ao usar a técnica de parada prematura de treinamento (early stopping), após 5 épocas sem melhora no parâmetro monitorado (neste caso a precisão de treinamento, val_accuracy), o treinamento é interrompido. Observamos que o MLP apresenta uma acurácia de cerca de 30%. Conforme esperado ele não resolve bem o problema de classificação.

In [3]:
model_MLP = tf.keras.Sequential([ 
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(256, activation='relu'),   #Camada escondida 1
    tf.keras.layers.Dense(256, activation='relu'),   #Camada escondida 2
    tf.keras.layers.Dense(5, activation='softmax')  #Camada de saída
])   

model_MLP.compile(
    optimizer=tf.keras.optimizers.Adam(),   #Otimizador Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model_MLP.fit(
    train_ds, 
    train_labels, 
    epochs=20, 
    validation_split=0.2, 
    batch_size=32, 
    callbacks=[es]
)

# Evaluate the model on the test dataset
loss, accuracy = model_MLP.evaluate(test_ds, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 10s 127ms/step - accuracy: 0.2551 - loss: 839.1895 - val_accuracy: 0.3288 - val_loss: 203.3070
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.3397 - loss: 120.3524 - val_accuracy: 0.3191 - val_loss: 100.4006
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.3711 - loss: 52.3478 - val_accuracy: 0.3249 - val_loss: 49.7330
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.3389 - loss: 24.4813 - val_accuracy: 0.2549 - val_loss: 15.7129
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.2857 - loss: 9.3323 - val_accuracy: 0.2529 - val_loss: 2.9597
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.2670 - loss: 1.9715 - val_accuracy: 0.2412 - val_loss: 2.1306
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3015 - loss: 743.2046
Accuracy: 0.30245232582092285


### 2.1 Testando técnicas de normalização
Vamos explorar técnicas como normalização L1,L2 e dropout, avaliando seu impacto no desempenho

In [9]:
# Regularização L1

model_MLP = tf.keras.Sequential([ 
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l1(0.01)),   #Camada escondida 1
    tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=regularizers.l1(0.01)),   #Camada escondida 2
    tf.keras.layers.Dense(5, activation='softmax')  #Camada de saída
])   

model_MLP.compile(
    optimizer=tf.keras.optimizers.Adam(),   #Otimizador Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model_MLP.fit(
    train_ds, 
    train_labels, 
    epochs=10, 
    validation_split=0.2, 
    batch_size=32,
    callbacks=[es]

)

# Evaluate the model on the test dataset
loss, accuracy = model_MLP.evaluate(test_ds, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 13s 172ms/step - accuracy: 0.2424 - loss: 1182.6146 - val_accuracy: 0.3424 - val_loss: 411.0271
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 174ms/step - accuracy: 0.3560 - loss: 335.5598 - val_accuracy: 0.3405 - val_loss: 231.2623
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 168ms/step - accuracy: 0.4368 - loss: 196.8247 - val_accuracy: 0.3171 - val_loss: 212.4454
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 168ms/step - accuracy: 0.3740 - loss: 165.6553 - val_accuracy: 0.3327 - val_loss: 170.8374
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 170ms/step - accuracy: 0.4177 - loss: 162.5100 - val_accuracy: 0.3735 - val_loss: 151.5728
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 173ms/step - accuracy: 0.3866 - loss: 153.4328 - val_accuracy: 0.3502 - val_loss: 200.1851
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 174ms/step - accuracy: 0.3534 - loss: 166.5182 - val_accuracy: 0.2879 - val_loss: 170.9317
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 168ms/step - accuracy: 0.3

In [10]:
# Regularização L2

model_MLP = tf.keras.Sequential([ 
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),   #Camada escondida 1
    tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.01)),   #Camada escondida 2
    tf.keras.layers.Dense(5, activation='softmax')  #Camada de saída
])   

model_MLP.compile(
    optimizer=tf.keras.optimizers.Adam(),   #Otimizador Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model_MLP.fit(
    train_ds, 
    train_labels, 
    epochs=10, 
    validation_split=0.2, 
    batch_size=32,
    callbacks=[es]

)

# Evaluate the model on the test dataset
loss, accuracy = model_MLP.evaluate(test_ds, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 12s 170ms/step - accuracy: 0.2541 - loss: 812.4495 - val_accuracy: 0.3366 - val_loss: 110.8457
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 12s 178ms/step - accuracy: 0.3642 - loss: 73.6263 - val_accuracy: 0.3794 - val_loss: 31.3835
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 170ms/step - accuracy: 0.3739 - loss: 32.9841 - val_accuracy: 0.3444 - val_loss: 34.4912
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 170ms/step - accuracy: 0.4015 - loss: 21.3660 - val_accuracy: 0.3444 - val_loss: 26.2302
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 174ms/step - accuracy: 0.4119 - loss: 15.1536 - val_accuracy: 0.3988 - val_loss: 15.5691
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 167ms/step - accuracy: 0.4567 - loss: 12.9904 - val_accuracy: 0.2218 - val_loss: 21.7726
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 166ms/step - accuracy: 0.3244 - loss: 20.1618 - val_accuracy: 0.2763 - val_loss: 19.9823
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 167ms/step - accuracy: 0.2794 - loss: 1

In [12]:
# Dropout

model_MLP = tf.keras.Sequential([ 
    tf.keras.layers.Flatten(),  
    
    tf.keras.layers.Dense(256, activation='relu'),   #Camada escondida 1
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(256, activation='relu'),   #Camada escondida 2
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(5, activation='softmax')  #Camada de saída
])   

model_MLP.compile(
    optimizer=tf.keras.optimizers.Adam(),   #Otimizador Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model_MLP.fit(
    train_ds, 
    train_labels, 
    epochs=10, 
    validation_split=0.2, 
    batch_size=32,
    callbacks=[es]

)

# Evaluate the model on the test dataset
loss, accuracy = model_MLP.evaluate(test_ds, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 10s 132ms/step - accuracy: 0.2040 - loss: 1177.5011 - val_accuracy: 0.2490 - val_loss: 1.8138
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 129ms/step - accuracy: 0.2528 - loss: 1.9383 - val_accuracy: 0.2510 - val_loss: 1.6345
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 128ms/step - accuracy: 0.2456 - loss: 1.6849 - val_accuracy: 0.2510 - val_loss: 1.6375
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 127ms/step - accuracy: 0.2387 - loss: 1.6732 - val_accuracy: 0.2510 - val_loss: 1.6357
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 130ms/step - accuracy: 0.2377 - loss: 1.6541 - val_accuracy: 0.2510 - val_loss: 1.6367
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 128ms/step - accuracy: 0.2170 - loss: 1.6093 - val_accuracy: 0.2510 - val_loss: 1.6370
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 129ms/step - accuracy: 0.2575 - loss: 1.6000 - val_accuracy: 0.2510 - val_loss: 1.6368
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.2575 - loss: 4.3174
Accuracy: 0.23160763084888

## 3. Uso da rede VGG16 pré treinada
Vamos utilizar a rede VGG16 como pré processamento fixo das imagens. Ainda que essa rede tenha sido treinada em um conjunto de dados muito mais amplo do que somente flores, vamos verificar que podemos reusar esse treinamento para aumentar a performance do nosso problema de classificação.

### 3.1 Processamento da rede VGG
Carregamos o modelo VGG, pre processamos os dados e checamos se o formato deles está em tensores condizentes com o MLP

In [15]:
# Carregando modelo VGG16
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

# Dados são processados de maneira semelhante ao treinamento original da VGG16.
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

# Checando se o formato dos dados esta consistente com o MLP
print(train_ds.shape)
print(test_ds.shape)

# Resumo da arquitetura do modelo
base_model.summary()

(2569, 150, 150, 3)
(1101, 150, 150, 3)


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

### 3.2 Treinamento do MLP que sucede a VGG
Ele ta overfitting, usar regularização. 

In [ ]:
model_MLP_from_VGG = tf.keras.Sequential([ 
    base_model,                                     #Base do VGG
    tf.keras.layers.Flatten(),  
    
    tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(5, activation='softmax')  #Camada de saída
])   

model_MLP_from_VGG.compile(
    optimizer=tf.keras.optimizers.Adam(),   #Otimizador Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model_MLP_from_VGG.fit(
    train_ds, 
    train_labels, 
    epochs=15, 
    validation_split=0.2, 
    batch_size=32, 
    callbacks=[es]

)

# Evaluate the model on the test dataset
loss, accuracy = model_MLP_from_VGG.evaluate(test_ds, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.2225 - loss: 9.0306 - val_accuracy: 0.3035 - val_loss: 3.8257
Epoch 2/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.2709 - loss: 4.6335 - val_accuracy: 0.3813 - val_loss: 3.5519
Epoch 3/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.3081 - loss: 3.6453 - val_accuracy: 0.3949 - val_loss: 3.2768
Epoch 4/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.3267 - loss: 3.3808 - val_accuracy: 0.4650 - val_loss: 2.9612
Epoch 5/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 160s 2s/step - accuracy: 0.3503 - loss: 3.1190 - val_accuracy: 0.4514 - val_loss: 2.8043
Epoch 6/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.3675 - loss: 2.9011 - val_accuracy: 0.4747 - val_loss: 2.6127
Epoch 7/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.3749 - loss: 2.6613 - val_accuracy: 0.5389 - val_loss: 2.3858
Epoch 8/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - accuracy: 0.3727 - loss: 2.5682 - val_accuracy: 0.5117 - v

## 4. Erros de classificação e Matriz de confusão